##  Example Weather Prediction

**Status:** Public Availability <br>

**Author:** Jaume Manero<br>
**Date created:** 10/1/2022<br>
**Last modified:** 10/4/2023<br>
**Description:** Weather prediction example with Random Forest Direct Regression, <br>
                    meaning one regression per each ahead value

In [1]:
# We analyze the contents of the data to get some inspiration for the forecasting program
# The strategy is Direct Regression, this means one regression per ahead value 

In [2]:
# Import libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from windrose import WindroseAxes


import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, GaussianNoise, Input, BatchNormalization, ELU
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

2023-04-10 23:49:24.174348: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-10 23:49:24.216270: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-10 23:49:24.217150: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 23:49:24.983532: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#Import data
file = './YHZ_hourly/YHZ_hourly.csv'

df1 = pd.read_csv(file)

/tmp/ipykernel_6349/1984970438.py:4: DtypeWarning: Columns (12,14,20,22,24,26,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file)


In [4]:
# we create a datetime index
df1['date'] = pd.to_datetime(df1['LOCAL_DATE'])
df1 = df1.sort_values('date')
df1 = df1.reset_index(drop=True)

In [5]:
# Downsample series to average temp per day
df2 = df1 [['date', 'TEMP']]
df_day_grouped = df2.set_index('date').groupby(pd.Grouper(freq='d')).agg({'TEMP': ['mean', 'min', 'max']})
df_day_grouped = df_day_grouped.reset_index()
df_day_grouped.columns = df_day_grouped.columns.map(''.join)

=======================================================
##### Data Matrix Preparation 
We transform the file in as sequence of examples \
(1) 1 2 3 4 5 6 7 8 9 10 -> lag 2 \
(1) 12, 23, 34, 45, 56, 67, 78, 89, 910 \
(2) 1 2 3 4 5 6 7 8 9 10 -> lag 3  \
(2) 123, 234, 345, 456, 567, 678, 789, 8910,  \ 

In [6]:
df1

,x,y,STATION_NAME,CLIMATE_IDENTIFIER,ID,LOCAL_DATE,PROVINCE_CODE,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,...,STATION_PRESSURE_FLAG,VISIBILITY,VISIBILITY_FLAG,WINDCHILL,WINDCHILL_FLAG,WIND_DIRECTION,WIND_DIRECTION_FLAG,WIND_SPEED,WIND_SPEED_FLAG,date
0,-63.500014,44.880017,HALIFAX STANFIELD INT'L A,8202250,8202250.1961.1.3.6,1961-01-03 06:00:00,NS,1961,1,3,...,NaN,16.1,NaN,-9.0,NaN,25.0,NaN,8.0,NaN,1961-01-03 06:00:00
1,-63.500014,44.880017,HALIFAX STANFIELD INT'L A,8202250,8202250.1961.1.3.7,1961-01-03 07:00:00,NS,1961,1,3,...,NaN,16.1,NaN,-11.0,NaN,29.0,NaN,10.0,NaN,1961-01-03 07:00:00
2,-63.500014,44.880017,HALIFAX STANFIELD INT'L A,8202250,8202250.1961.1.3.8,1961-01-03 08:00:00,NS,1961,1,3,...,NaN,16.1,NaN,-11.0,NaN,25.0,NaN,10.0,NaN,1961-01-03 08:00:00
3,-63.500014,44.880017,HALIFAX STANFIELD INT'L A,8202250,8202250.1961.1.3.9,1961-01-03 09:00:00,NS,1961,1,3,...,NaN,16.1,NaN,-11.0,NaN,29.0,NaN,10.0,NaN,1961-01-03 09:00:00
4,-63.500014,44.880017,HALIFAX STANFIELD INT'L A,8202250,8202250.1961.1.3.10,1961-01-03 10:00:00,NS,1961,1,3,...,NaN,16.1,NaN,-10.0,NaN,29.0,NaN,8.0,NaN,1961-01-03 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379361,-63.508611,44.881111,HALIFAX STANFIELD INT'L A,8202251,8202251.2022.7.9.19,2022-07-09 19:00:00,NS,2022,7,9,...,NaN,24.1,NaN,NaN,NaN,32.0,NaN,19.0,NaN,2022-07-09 19:00:00
379362,-63.508611,44.881111,HALIFAX STANFIELD INT'L A,8202251,8202251.2022.7.9.20,2022-07-09 20:00:00,NS,2022,7,9,...,NaN,24.1,NaN,NaN,NaN,32.0,NaN,14.0,NaN,2022-07-09 20:00:00
379363,-63.508611,44.881111,HALIFAX STANFIELD INT'L A,8202251,8202251.2022.7.9.21,2022-07-09 21:00:00,NS,2022,7,9,...,NaN,24.1,NaN,NaN,NaN,27.0,NaN,10.0,NaN,2022-07-09 21:00:00
379364,-63.508611,44.881111,HALIFAX STANFIELD INT'L A,8202251,8202251.2022.7.9.22,2022-07-09 22:00:00,NS,2022,7,9,...,NaN,24.1,NaN,NaN,NaN,27.0,NaN,14.0,NaN,2022-07-09 22:00:00


In [7]:
# Data Matrix Preparation


df1.columns
cols = ['x', 'y', 'STATION_NAME', 'CLIMATE_IDENTIFIER', 'ID', 'LOCAL_DATE',
       'PROVINCE_CODE', 'LOCAL_YEAR', 'LOCAL_MONTH', 'LOCAL_DAY', 'LOCAL_HOUR',
       'TEMP', 'TEMP_FLAG', 'DEW_POINT_TEMP', 'DEW_POINT_TEMP_FLAG', 'HUMIDEX',
       'HUMIDEX_FLAG', 'PRECIP_AMOUNT', 'PRECIP_AMOUNT_FLAG',
       'RELATIVE_HUMIDITY', 'RELATIVE_HUMIDITY_FLAG', 'STATION_PRESSURE',
       'STATION_PRESSURE_FLAG', 'VISIBILITY', 'VISIBILITY_FLAG', 'WINDCHILL',
       'WINDCHILL_FLAG', 'WIND_DIRECTION', 'WIND_DIRECTION_FLAG', 'WIND_SPEED',
       'WIND_SPEED_FLAG', 'date']

drop_cols = ['x', 'y', 'STATION_NAME', 'CLIMATE_IDENTIFIER', 'ID', 'LOCAL_DATE',
       'PROVINCE_CODE', 'LOCAL_YEAR', 'LOCAL_MONTH', 'LOCAL_DAY', 'LOCAL_HOUR']

ts_multivar = df1.drop(drop_cols, axis=1)

Nan_cols = ['DEW_POINT_TEMP_FLAG', 'HUMIDEX', 'HUMIDEX_FLAG', 'PRECIP_AMOUNT', 'STATION_PRESSURE_FLAG',
           'PRECIP_AMOUNT_FLAG', 'RELATIVE_HUMIDITY_FLAG', 'VISIBILITY_FLAG', 'WINDCHILL_FLAG',
           'WIND_DIRECTION_FLAG', 'WIND_SPEED_FLAG', 'WINDCHILL', 'TEMP_FLAG']
ts_multivar = ts_multivar.drop(Nan_cols, axis=1)


In [8]:
ts_multivar.isna().sum().sum()
ts_multivar = ts_multivar.dropna()
ts_multivar_original = ts_multivar

In [9]:
ts_multivar = ts_multivar.drop('date', axis=1)
weather_data = ts_multivar.to_numpy()
weather_data.shape

(378830, 7)

In [10]:
#===================
lag = 48
ahead = 24
#====================

# Examples dataframe creation
#print('Input dataframe structure', ts_multivar.shape)

# create train, test, val (60%, 20%, 20%)
train_split = 0.6
test_split = 0.2

datasize =  weather_data.shape[0]
trainsize = int(datasize * train_split)
half_test = int(datasize * test_split)
testsize = int(half_test*2)

sc=StandardScaler()
tmpdata = sc.fit_transform(weather_data)
Scaler = sc.fit(weather_data[:, 0].reshape(-1, 1))  # saves the scaler for the first variable for descaling
#Scaler = sc.fit(weather_data[:, 0])  # saves the scaler for the first variable for descaling
weather_data = tmpdata


In [11]:
t_s_multivariate = weather_data

def lag_matrix(wind_train, lag=lag, ahead=ahead):
    wind_train = data[:trainsize, :]
    lvect = []
    for i in range(lag + ahead):
       lvect.append(data[i: -lag - ahead + i, :])
    return(np.stack(lvect, axis=1))

# data = train -> test -> val

data = t_s_multivariate[:trainsize, :]

train = lag_matrix(data, lag, ahead)

train_x = train[:, :lag]
train_x = np.swapaxes(train_x, 1, 2)
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1] * train_x.shape[2]))

data  = t_s_multivariate[trainsize:trainsize + testsize, :]

test  = lag_matrix(data, lag, ahead)
half_test = int(test.shape[0] / 2)

test_x = test[:half_test, :lag]
val_x = test[half_test:, :lag]

train_y = train[:, -ahead:, 0]
test_y   = test[:half_test, -ahead:, 0]
val_y  = test[half_test:, -ahead:, 0]


test_x = np.swapaxes(test_x, 1, 2)
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1] * test_x.shape[2]))
val_x = np.swapaxes(val_x, 1, 2)
val_x = np.reshape(val_x, (val_x.shape[0], val_x.shape[1] * val_x.shape[2]))


print("=========== SHAPE INPUT-OUTPUT MATRIXES =====================")
print("shape x_training :", train_x.shape, " y_training: : ", train_y.shape)
print("shape x_test     : ", test_x.shape, " y_test      : ", test_y.shape)
print("shape x_val      : ", val_x.shape,  " y_val       : ", val_y.shape)
print("=============================================================")

input_dimensions = train_x.shape[1:]
print(input_dimensions)

=========== SHAPE INPUT-OUTPUT MATRIXES =====================
shape x_training : (227226, 336)  y_training: :  (227226, 24)
shape x_test     :  (75730, 336)  y_test      :  (75730, 24)
shape x_val      :  (75730, 336)  y_val       :  (75730, 24)
(336,)


In [12]:
# Random Forest regression
# Direct Regression. One regression per ahead value
from tqdm import trange

RF = RandomForestRegressor(n_estimators=50, max_features="auto", oob_score= True, random_state=0)

# Random Forest prediction

RF.fit(train_x, train_y[:,0])

val_yp = RF.predict(val_x) 
test_yp = RF.predict(test_x) 

for i in trange (1, ahead, desc="fit loop"):
   RF.fit(train_x, train_y[:,i])
   z1 = RF.predict(val_x) 
   z2 = RF.predict(test_x) 
   val_yp = np.column_stack((val_yp,z1))
   test_yp = np.column_stack((test_yp,z2))


/home/manero/DL/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


NameError: name 'trange' is not defined

In [ ]:
print('val_yp.shape', val_yp.shape, 'test_yp.shape', test_yp.shape)

In [ ]:
# Evaluating results

lresults = []
val_y_R2 = []
test_y_R2 = []
acum_val, acum_test = 0,0
for i in trange (1,ahead+1):
   v_yp = Scaler.inverse_transform(val_yp[:, i - 1].reshape(-1,1))
   v_y = Scaler.inverse_transform(val_y[:, i - 1].reshape(-1,1))
   t_yp = Scaler.inverse_transform(test_yp[:, i - 1].reshape(-1,1))
   t_y = Scaler.inverse_transform(test_y[:, i - 1].reshape(-1,1))
   val_y_R2.append(r2_score(v_yp,v_y))
   test_y_R2.append(r2_score(t_yp, t_y))
   
print ("Results === Test == Validation ")
print('=============================')
for i in range (0,ahead):
   print('step', "{:<4d}".format(i+1), '- ', "{:.3f}".format(val_y_R2[i]), "   {:.3f}".format(test_y_R2[i]))          
   acum_val = acum_val + val_y_R2[i]
   acum_test = acum_test + test_y_R2[i]
print('=============================')
print('Summary', '  - ', "{:.3f}".format(acum_val), "   {:.3f}".format(acum_test))

In [ ]:
v_yp.shape

In [ ]:
#plot prediction accuracy decay
# 
N = 24
d = [x for x in range(N)]

fig, ax = plt.subplots(figsize=(30,15))

ax.plot(d,val_y_R2,color='green',linewidth=3.0)
ax.plot(d,test_y_R2,color='red',linewidth=3.0)
ax.legend(["R2 Validation", "R2 Test"])
plt.title("Accuracy Decay", size=22)
plt.ylabel("R2", size=18)
plt.xlabel("Hours Ahead", size=18)
plt.grid(axis='both')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.show()

In [ ]:
#plot prediction and ground truth
# Let's do it for the last 100 days
N = 240
d = [x for x in range(N)]
d1 = ts_multivar_original.iloc[-N:]

df_temps = df_day_grouped[df_day_grouped['date'].dt.year == 2021 ]
fig, ax = plt.subplots(figsize=(30,15))

ax.plot(d1['date'],v_y[-N:,],color='green',linewidth=3.0, )
ax.plot(d1['date'],v_yp[-N:,],color='red',linewidth=3.0, linestyle='--')
ax.legend(["real temperature", "predicted temperature (24h ahead)"])
plt.title("10 days Temp Prediction in Standsed Airport HALIFAX" in "+str(y)", size=22)
plt.ylabel("TEMP (degrees)", size=18)
plt.xlabel("date", size=18)
plt.grid(axis='both')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.show()

In [ ]:
df = d1

plt.figure(figsize=(20,10))
plt.rc('legend', fontsize=20)
df.plot(subplots=True, legend=True, figsize=(22,30), use_index=True, fontsize=20)
plt.show()

In [ ]:
import session_info
session_info.show(html=False)

In [ ]:
#EOF